# Beyin Tümörü Sınıflandırma (MRI) - CNN Projesi

Bu notebook, MRI görüntülerini kullanarak beyin tümörü tespiti yapan bir Derin Öğrenme modelini içermektedir.

**Hedef:** 4 farklı sınıfı (Glioma, Meningioma, No Tumor, Pituitary) yüksek doğrulukla sınıflandırmak.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow Versiyonu:", tf.__version__)

## 1. Veri Hazırlama ve Loaders

Veri setini yüklüyoruz ve Data Augmentation uyguluyoruz.

In [ ]:
TRAIN_DIR = 'archive/Training'
TEST_DIR = 'archive/Testing'
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Eğitim ve Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

## 2. Model Mimarisinin Oluşturulması

4 katmanlı Conv2D içeren CNN yapısı.

In [ ]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 3. Model Eğitimi

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("Eğitim başlıyor...")
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stop]
)

## 4. Sonuçların Görselleştirilmesi

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Eğitim Başarısı')
    plt.plot(epochs, val_acc, 'ro-', label='Validasyon Başarısı')
    plt.title('Doğruluk (Accuracy)')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Eğitim Kaybı')
    plt.plot(epochs, val_loss, 'ro-', label='Validasyon Kaybı')
    plt.title('Kayıp (Loss)')
    plt.legend()
    plt.show()

plot_history(history)

## 5. Değerlendirme ve Karmaşıklık Matrisi

In [ ]:
print("Test verisi değerlendiriliyor...")
class_names = list(test_generator.class_indices.keys())
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('Gerçek')
plt.xlabel('Tahmin')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Modeli kaydet
model.save('brain_tumor_model.keras')
print("Model kaydedildi.")